# Periodic homogenization of linear elastic materials
## Introduction
This tour will show how to perform periodic homogenization of linear elastic materials. The considered 2D plane strain problem deals with a skewed unit cell of dimensions $1\times\frac{\sqrt{3}}{2}$ consisting of circular inclusions (markered **1**) of radius $R$ with elastic properties $(E_r,\nu_r)$ and embedded in a matrix material (marked **0**) of properties $(E_m,\nu_m)$ following an hexagonal pattern (see Figure 1). A classical result of homogenization theory ensures that the resulting overall behavior will be isotropic, a property that will be numerically verified later.

We suggest reading first the [2D linear elasticity](https://github.com/kutsjuice/FEniCSx-tutorials/tree/main/2D%20linear%20elasticity) tour if you are not familiar with implementation of elastic materials.
<table>
    <tr>
        <td>
            <div style="text-align:center">
              <img src="pics/pattern.png" alt="Material patern" width=400 height=400 title="Problem geometry">
              <figcaption><b>Material patern</b></figcaption>
            </div>
        </td>
        <td>
            <td>
                <div style="text-align:center">
                  <img src="pics/RVE.png" alt="Representive volume element" width=400 height=400 title="Problem geometry">
                  <figcaption><b>Representive volume element</b></figcaption>
                </div>
        </td>
    </tr>
</table> 

In [1]:
import numpy as np
import ufl

from mpi4py import MPI
from petsc4py.PETSc import ScalarType

from dolfinx import mesh, fem, plot, io
from dolfinx.io import XDMFFile, gmshio
from dolfinx.mesh import DiagonalType
import gmsh

import pyvista
pyvista.set_jupyter_backend('pythreejs');
# pyvista.global_theme.trame.server_proxy_enabled = True;
# pyvista.global_theme.trame.server_proxy_prefix = '/proxy/';


  pyvista.set_jupyter_backend('pythreejs');



Generation of domain is performed using gmsh.
To apply periodic boundary conditions we need to create a mesh with perfectly matching vertices on opposite boundaries.
<div style="text-align:center">
  <img src="pics/geometry.png" alt="Governing geometry" width=500 height=500 title="Problem geometry">
  <figcaption><b>Governing geometry</b></figcaption>
</div>

In [2]:
a = 1.;
R = 0.2;
t = np.pi/3;
b = np.sin(t)*a;
bR = np.sin(t)*R;
c = a*np.cos(t);
cR = R*np.cos(t);

mesh_size = 0.01;

nx = int(21);
nR = int(R*nx);
ns = int(nx-2*nR);

# SHOW_PYVISTA = False;
SHOW_PYVISTA = True;

gmsh.initialize();
model = gmsh.model();
model.add("main_domain");
model.setCurrent("main_domain");

# Choose if Gmsh output is verbose
gmsh.option.setNumber("General.Terminal", 0)

try:
    # first 4 points are corner points
    model.occ.add_point(0, 0, 0, mesh_size, 1);
    model.occ.add_point(a, 0, 0, mesh_size, 2);
    model.occ.add_point(a+c, b, 0, mesh_size, 3);
    model.occ.add_point(c, b, 0, mesh_size, 4);
    
    # following points are pointss of intersection of circles and edges
    model.occ.add_point(R, 0, 0, mesh_size, 5);
    model.occ.add_point(a-R, 0, 0, mesh_size, 6);
    model.occ.add_point(a+cR, bR, 0, mesh_size, 7);
    model.occ.add_point(a+c-cR, b-bR, 0, mesh_size, 8);
    model.occ.add_point(a+c-R, b, 0, mesh_size, 9);
    model.occ.add_point(c+R, b, 0, mesh_size, 10);
    model.occ.add_point(c-cR, b-bR, 0, mesh_size, 11);
    model.occ.add_point(cR, bR, 0, mesh_size, 12);
    
    model.occ.add_line(1, 5, 1);
    model.occ.add_line(5, 6, 2);
    model.occ.add_line(6, 2, 3);
    model.occ.add_line(2, 7, 4);
    model.occ.add_line(7, 8, 5);
    model.occ.add_line(8, 3, 6);
    model.occ.add_line(3, 9, 7);
    model.occ.add_line(9, 10, 8);
    model.occ.add_line(10, 4, 9);
    model.occ.add_line(4, 11, 10);
    model.occ.add_line(11, 12, 11);
    model.occ.add_line(12, 1, 12);
    
    model.occ.add_circle_arc(5, 1, 12, 13);
    model.occ.add_circle_arc(7, 2, 6, 14);
    model.occ.add_circle_arc(9, 3, 8, 15);
    model.occ.add_circle_arc(11, 4, 10, 16);
    
    # if line should be added to the loop in reversed order, than it's number should be negative
    matrix_loop = model.occ.add_curve_loop([2, -14, 5, -15, 8, -16, 11, -13]);
    
    lb_circle_loop = model.occ.add_curve_loop([1, 13, 12]);
    rb_circle_loop = model.occ.add_curve_loop([3, 4, 14]);
    rt_circle_loop = model.occ.add_curve_loop([6, 7, 15]);
    lt_circle_loop = model.occ.add_curve_loop([9, 10, 16]);
    
    model.occ.synchronize();
    
    matrix_plane = model.occ.add_plane_surface([matrix_loop]);
    lb_circle_plane = model.occ.add_plane_surface([lb_circle_loop]);
    rb_circle_plane = model.occ.add_plane_surface([rb_circle_loop]);
    rt_circle_plane = model.occ.add_plane_surface([rt_circle_loop]);
    lt_circle_plane = model.occ.add_plane_surface([lt_circle_loop]);
    
#     for curve in [1,3,4,6,7,9,10,12, 13, 15]:
#         model.mesh.set_transfinite_curve(curve, nR);
    
#     for curve in [2,5,8,11]:
#         model.mesh.set_transfinite_curve(curve, ns);
    
#     for curve in [14, 16]:
#         model.mesh.set_transfinite_curve(curve, 2*nR);
    left2right = [1, 0, 0, a, 
                  0, 1, 0, 0, 
                  0, 0, 1, 0, 
                  0, 0, 0, 1];
    bottom2top = [1, 0, 0, c,
                  0, 1, 0, b,
                  0, 0, 1, 0,
                  0, 0, 0, 0];
    model.mesh.set_periodic(1,
                            [4, 5, 6],
                            [12, 11, 10],
                            left2right);
    model.mesh.set_periodic(1, 
                            [9, 8, 7],
                            [1, 2, 3], 
                            bottom2top);
    
    model.occ.synchronize();
    model.add_physical_group(dim = 1, tags = [1, 2, 3], tag = 1);
    model.add_physical_group(dim = 1, tags = [4, 5, 6], tag = 2);
    model.add_physical_group(dim = 1, tags = [7, 8, 9], tag = 3);
    model.add_physical_group(dim = 1, tags = [10, 11, 12], tag = 4);
    model.add_physical_group(dim = 2, tags = [matrix_plane], tag = 0);
    model.add_physical_group(dim = 2, tags = [lb_circle_plane, 
                                              rb_circle_plane, 
                                              rt_circle_plane, 
                                              lt_circle_plane
                                             ], tag = 1);
    
    gmsh.option.setNumber("Mesh.Algorithm", 8);
    gmsh.option.setNumber('Mesh.RecombineAll', 1)
    gmsh.option.setNumber('Mesh.RecombinationAlgorithm', 2)
    model.mesh.generate(dim=2);
    # model.mesh.recombine();
    # Create a DOLFINx mesh (same mesh on each rank)
    msh, cell_markers, facet_markers = gmshio.model_to_mesh(model, MPI.COMM_SELF,0,gdim=2);
    msh.name = "Box";
    cell_markers.name = f"{msh.name}_cells";
    facet_markers.name = f"{msh.name}_facets";
finally:
    gmsh.finalize();
    # pass
        
if SHOW_PYVISTA:
    
    pyvista.start_xvfb();
    plotter = pyvista.Plotter();

    topology, cell_types, geometry = plot.create_vtk_mesh(msh);
    grid = pyvista.UnstructuredGrid(topology, cell_types, geometry);
    actor_0 = plotter.add_mesh(grid, style="wireframe", color="k");
    plotter.show_axes()
    plotter.export_html('myplot.html')
    if not pyvista.OFF_SCREEN:
        plotter.show();
    else:
        figure = plotter.screenshot("fundamentals_mesh.png");

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…